In [1]:
import cobra
import cobrakbase

cobrakbase 0.2.7


In [20]:
kbase = cobrakbase.KBaseAPI()

In [2]:
modelseed_local = cobrakbase.modelseed.from_local('/Users/fliu/workspace/jupyter/ModelSEEDDatabase/')

In [3]:
from modelseedpy.core.biolog import BiologPlate, Biolog

In [5]:
modelseed_local.compound_aliases['cpd03198']['BiGG'] = {'melib'}
modelseed_local.compound_aliases['cpd00072']['BiGG'] = {'f6p'}

In [13]:
base_media = {
    'cpd00001', # H2O
    'cpd00009', # Pi
    'cpd00013', # NH4
    'cpd10515', # Fe+2
    'cpd10516', # Fe+3
    'cpd00030', # Mn2+
    'cpd00034', # Zn2+
    'cpd00048', # so4
    'cpd00058', # Cu2+
    'cpd00063', # Ca2+
    'cpd00067', # H+
    'cpd00099', # Cl-
    'cpd00149', # Co2+
    'cpd00205', # K+
    'cpd00244', # Ni2+
    'cpd00254', # Mg
    'cpd00971', # Na+
    'cpd11574', # Molybdate
}
base_media = list(map(lambda x: modelseed_local.get_seed_compound(x).id, base_media))

In [8]:
base_media_bigg = []
for k in base_media:
    a = modelseed_local.compound_aliases[k.id]['BiGG']
    #print(k.data['name'], a)
    if k.id == 'cpd00001':
        a = {'h2o'}
    if k.id == 'cpd00013':
        a = {'nh4'}
    base_media_bigg.append(list(a)[0])
base_media_bigg

['mn2',
 'zn2',
 'na1',
 'pi',
 'ni2',
 'h2o',
 'k',
 'fe3',
 'nh4',
 'h',
 'cl',
 'fe2',
 'mobd',
 'so4',
 'cu2',
 'ca2',
 'mg2',
 'cobalt2']

In [19]:
pm1_plate = BiologPlate('PM1', ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
for l in pm1_str.split('\n'):
    o = l.split('\t')
    well_id = o[0]
    desc = o[1]
    compounds = set() if len(o) < 3 else set(o[2].split(';'))
    compounds_final = set()
    for cpd_id in compounds:
        if len(cpd_id) > 1:
            compounds_final |= {cpd_id}
            continue
            aliases = modelseed_local.compound_aliases[cpd_id]
            if 'BiGG' in aliases and len(aliases['BiGG']) == 1:
                compounds_final |= aliases['BiGG']
    #print(compounds, compounds_final)
    pm1_plate.wells[well_id] = {
        'desc': desc,
        'compounds': compounds_final,
        'value': 10
    }
pm1_plate.add_base(base_media, 1000)
pm1_plate.get_media('A04')

{'cpd00030': 1000,
 'cpd00034': 1000,
 'cpd00971': 1000,
 'cpd00009': 1000,
 'cpd00244': 1000,
 'cpd00001': 1000,
 'cpd00205': 1000,
 'cpd10516': 1000,
 'cpd00013': 1000,
 'cpd00067': 1000,
 'cpd00099': 1000,
 'cpd10515': 1000,
 'cpd11574': 1000,
 'cpd00048': 1000,
 'cpd00058': 1000,
 'cpd00063': 1000,
 'cpd00254': 1000,
 'cpd00149': 1000,
 'cpd00571': 10}

In [21]:
model = kbase.get_from_ws('GCF_000005845.2.RAST.fbamodel', 77774)

In [379]:
model_id = 'GCF_000005845.2.RAST.fbamodel'
media_id = 'Carbon-D-Glucose'
media = kbase.get_from_ws(media_id, 77774)
kbmodel = kbase.get_object(model_id, 77774)
model = kbase.get_from_ws(model_id, 77774)
print(len(kbmodel['modelreactions']), len(model.reactions))
rxn = cobra.core.Reaction('CYTBO3_4pp', 'CYTBO3_4pp', '', 0, 1000)
rxn.add_metabolites({
    model_gf.metabolites.cpd15561_c0: -1,
    model_gf.metabolites.cpd15560_c0:  1,
    model_gf.metabolites.cpd00067_e0: -4,
    model_gf.metabolites.cpd00067_c0:  4,
    model_gf.metabolites.cpd00001_c0:  1,
    model_gf.metabolites.cpd00007_c0:  -0.5,
})
model.add_reaction(rxn)
model.reactions.rxn00452_c0

1599 1777


Reaction identifier,rxn00452_c0
Name,S-Adenosyl-L-methionine:L-homocysteine S-methyltransferase_c0
Memory address,0x07faae4a678d0
Stoichiometry,cpd00017_c0 + cpd00135_c0 --> cpd00019_c0 + cpd00060_c0 + cpd00067_c0 S-Adenosyl-L-methionine_c0 + Homocysteine_c0 --> S-Adenosyl-homocysteine_c0 + L-Methionine_c0 + H+_c0
GPR,(b0261)
Lower bound,0
Upper bound,1000


In [380]:
from cobrakbase.core.fba_utilities import KBaseFBAUtilities
from cobrakbase.flux_analysis import gapfill
from cobrakbase.flux_analysis.gapfilling import insert_gf, combine
utilities = KBaseFBAUtilities(model, kbmodel, kbase, media = media,
                              default_uptake = 100,
                              default_excretion = 100,
                              blacklist = [])
utilities.cobramodel.medium = media

In [381]:
utilities.default_uptake = 0
penalties = utilities.build_model_extended_for_gapfilling(1,[],[],2,{})
utilities.set_objective_from_target_reaction('bio1', 1)
utilities.convert_objective_to_constraint(0.1, None)
utilities.create_minimal_reaction_objective(penalties, default_penalty = 0)
gapfilling_solution = model.optimize()
output_1 = utilities.compute_gapfilled_solution(penalties)
flux_values = utilities.binary_check_gapfilling_solution(penalties, 1)
output_2 = utilities.compute_gapfilled_solution(penalties, flux_values)

0.32648922734328056

In [227]:
import math
def get_net(rxns, model, s):
    net = {}
    l = set(map(lambda x: x.id, model.reactions))
    #rxns = ['rxn39859_c', 'rxn39860_c', 'rxn07589_c', 'rxn07588_c']
    for r in rxns:
        if r in l:
            mrxn = model.reactions.get_by_id(r)
            if mrxn.id in s.fluxes:
                v = s.fluxes[mrxn.id]
                for m in mrxn.metabolites:
                    if not m.id in net:
                        net[m.id] = 0
                    net[m.id] += (mrxn.metabolites[m] * v)
            
    for o in net:
        if not net[o] == 0 and math.fabs(net[o]) < 1e-10:
            #print(o, net[o])
            #net[o] = 0
            pass
    for o in net:
        m = model.metabolites.get_by_id(o)
        if net[o] > 0:
            print('++', m.id, m.name, net[o])
    for o in net:
        m = model.metabolites.get_by_id(o)
        if net[o] < 0:
            print('--', m.id, m.name, net[o])
    return net

In [218]:
l = set(map(lambda x: x.id, model_gf.reactions))
from cobra.core import Solution
import pandas as pd
solution_data = {}
for rxn_id in flux_values:
    r = flux_values[rxn_id]['reverse']
    f = flux_values[rxn_id]['forward']
    if r != 0.0:
        solution_data[rxn_id] = -1 * r
    if f != 0.0:
        solution_data[rxn_id] = f
ser = pd.Series(data=solution_data)
solution = Solution(0, 'yay', ser)

In [230]:
#net = get_net(l, model_gf, solution)

In [262]:
for rxn_id in flux_values:
    cpd = modelseed_local.get_seed_compound(rxn_id[3:-3])
    r = flux_values[rxn_id]['reverse']
    if r != 0.0 and rxn_id.startswith('EX_'):
        print('R', rxn_id, r, cpd.name)
for rxn_id in flux_values:
    cpd = modelseed_local.get_seed_compound(rxn_id[3:-3])
    f = flux_values[rxn_id]['forward']
    if f != 0.0 and rxn_id.startswith('EX_'):
        print('F', rxn_id, f, cpd.name)

R EX_cpd00013_e0 0.8583604904712583 NH3
R EX_cpd00048_e0 0.022639718807314583 Sulfate
R EX_cpd00205_e0 0.00028061591595913103 K+
R EX_cpd00007_e0 0.002034465390703801 O2
R EX_cpd00030_e0 0.00028061591595913103 Mn2+
R EX_cpd10515_e0 0.0011224636638365241 Fe+2
R EX_cpd00099_e0 0.00028061591595913103 Cl-
R EX_cpd00027_e0 5.0 D-Glucose
R EX_cpd00034_e0 0.00028061591595913103 Zn2+
R EX_cpd00063_e0 0.00028061591595913103 Ca2+
R EX_cpd00254_e0 0.00028061591595913103 Mg
R EX_cpd00149_e0 0.00028061591595913103 Co2+
R EX_cpd00058_e0 0.00028061591595913103 Cu2+
R EX_cpd00009_e0 0.16251244516052166 Phosphate
F EX_cpd00139_e0 0.0008418477478773813 Glycolate
F EX_cpd00100_e0 0.0031145344943067604 Glycerol
F EX_cpd00029_e0 2.4409056585617375 Acetate
F EX_cpd00011_e0 0.01531295155636725 CO2
F EX_cpd00001_e0 0.384260331462563 H2O
F EX_cpd00794_e0 0.3401016505186795 TRHL
F EX_cpd00588_e0 2.2567353161851686 Sorbitol
F EX_cpd00235_e0 0.08125622258026083 D-mannose-6-phosphate
F EX_cpd00047_e0 2.94740979319

In [47]:
print(len(kbmodel['modelreactions']), len(model.reactions))
model.reactions.rxn00452_c0

1599 9951


Reaction identifier,rxn00452_c0
Name,S-Adenosyl-L-methionine:L-homocysteine S-methyltransferase_c0
Memory address,0x07fac74df3750
Stoichiometry,cpd00017_c0 + cpd00135_c0 <=> cpd00019_c0 + cpd00060_c0 + cpd00067_c0 S-Adenosyl-L-methionine_c0 + Homocysteine_c0 <=> S-Adenosyl-homocysteine_c0 + L-Methionine_c0 + H+_c0
GPR,(b0261)
Lower bound,-100
Upper bound,1000


In [ ]:
template_json = kbase.get_object(model_json_gf['template_ref'], None)

In [390]:
def make_rxn(i, lb, ub, m):
    rxn = cobra.core.Reaction(i, i, '', lb, ub)
    rxn.add_metabolites(m)
    return rxn

In [ ]:
model_gf.add_reaction(make_rxn('CYTBO3_4pp', 0, 1000, {
    model_gf.metabolites.cpd15561_c0: -1,
    model_gf.metabolites.cpd15560_c0:  1,
    model_gf.metabolites.cpd00067_e0: -4,
    model_gf.metabolites.cpd00067_c0:  4,
    model_gf.metabolites.cpd00001_c0:  1,
    model_gf.metabolites.cpd00007_c0:  -0.5,
}))

In [403]:
model_gf = insert_gf(kbmodel, output_2)
model_gf.reactions.rxn10042_c0.lower_bound = 0
model_gf.reactions.rxn10042_c0.upper_bound = 0

model_gf.add_reaction(make_rxn('FRD3', 0, 1000, {
    model_gf.metabolites.cpd00106_c0: -1,
    model_gf.metabolites.cpd00036_c0:  1,
    model_gf.metabolites.cpd15353_c0: -1,
    model_gf.metabolites.cpd15352_c0:  1,
}))
model_gf.medium = media
solution = cobra.flux_analysis.pfba(model_gf)
#model_gf.summary()
#solution.fluxes.to_csv('output_2_gf.csv')

In [374]:

#model_gf.add_reaction()

In [372]:
solution.fluxes.to_csv('output_2_fix_gf.csv')

In [323]:
cobra.io.save_json_model(model_gf, 'e_coli_kbase_dec_20.json')

In [396]:
model_gf.reactions.rxn08976_c0

Reaction identifier,rxn08976_c0
Name,quinone reductase_c0
Memory address,0x07fab4d016c10
Stoichiometry,cpd00004_c0 + cpd00067_c0 + cpd15352_c0 --> cpd00003_c0 + cpd15353_c0 NADH_c0 + H+_c0 + 2-Demethylmenaquinone 8_c0 --> NAD_c0 + 2-Demethylmenaquinol 8_c0
GPR,(b1109)
Lower bound,0
Upper bound,1000


In [404]:
model_gf.objective = 'rxn08173_c0'
solution = model_gf.optimize()
solution

,fluxes,reduced_costs
rxn05561_c0,0.0,0.000000
rxn00935_c0,0.0,0.000000
rxn00247_c0,0.0,-1.428571
rxn05625_c0,0.0,0.000000
rxn02342_c0,0.0,-0.285714
...,...,...
EX_cpd00122_e0,0.0,-4.857143
SK_cpd11416_c0,-0.0,0.000000
SK_cpd02701_c0,-0.0,0.000000
SK_cpd15302_c0,-0.0,0.000000


In [191]:
l = set(map(lambda x: x.id, model_gf.reactions))
for r in model_gf_kbase.reactions:
    if r.id not in l:
        print(r.id)

rxn00427_c0
rxn05293_c0
rxn02185_c0
rxn03436_c0
rxn12376_c0
rxn12868_c0
rxn05464_c0
rxn00125_c0
rxn09310_c0
EX_cpd15378_e0


In [286]:
output ={'reversed': {'rxn00452_c0': '='},
 'new': {
     'rxn10336_c0': '=', 
     'rxn08618_c0': '=', 
     'rxn00427_c0': '=',
     'rxn05293_c0': '=',
     'rxn02185_c0': '=',
     'rxn03436_c0': '=',
     'rxn12376_c0': '=',
     'rxn12868_c0': '=',
     'rxn05464_c0': '=',
     'rxn00125_c0': '=',
     'rxn09310_c0': '=',
    }
}

In [129]:
model_gf_kbase = kbase.get_from_ws('GCF_000005845.2.Glc.gf', 77774)
model_gf_kbase.medium = media
model_gf_kbase.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00007_e0,EX_cpd00007_e0,0.003335,0,0.00%
cpd00009_e0,EX_cpd00009_e0,0.1332,0,0.00%
cpd00013_e0,EX_cpd00013_e0,1.581,0,0.00%
cpd00027_e0,EX_cpd00027_e0,5,6,100.00%
cpd00030_e0,EX_cpd00030_e0,0.00046,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.00046,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.03895,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.00046,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.00046,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.00046,0,0.00%


In [ ]:
rxn00452_c0	<	(1) S-Adenosyl-L-methionine_c0[c0] + (1) Homocysteine_c0[c0] <=> (1) S-Adenosyl-homocysteine_c0[c0] + (1) L-Methionine_c0[c0] + (1) H+_c0[c0]	reversed

rxn10336_c0	>	(2) Phosphatidylglycerol dioctadecanoyl_c0[c0] => (1) Glycerol_c0[c0] + (1) Stearoylcardiolipin (B. subtilis)_c0[c0]	added
rxn08618_c0	>	(1) UDP-glucose_c0[c0] + (1) inner core oligosaccharide lipid A_c0[c0] => (1) UDP_c0[c0] + (4) H+_c0[c0] + (1) glucosyl-inner core oligosaccharide lipid A_c0[c0]	added
rxn00427_c0	>	(1) H2O2_c0[c0] + (1) Methanol_c0[c0] => (2) H2O_c0[c0] + (1) Formaldehyde_c0[c0]	added
rxn05293_c0	>	(1) NADH_c0[c0] + (1) H+_c0[c0] + (1) 2-C-methyl-D-erythritol2-4-cyclodiphosphate_c0[c0] => (1) H2O_c0[c0] + (1) NAD_c0[c0] + (1) 1-Hydroxy-2-methyl-2-butenyl 4-diphosphate_c0[c0]	added
rxn02185_c0	<	(1) TPP_c0[c0] + (1) ALCTT_c0[c0] <= (1) Pyruvate_c0[c0] + (1) 2-Hydroxyethyl-ThPP_c0[c0]	added
rxn03436_c0	>	(1) 2-Aceto-2-hydroxybutanoate_c0[c0] => (1) (R)-3-Hydroxy-3-methyl-2-oxopentanoate_c0[c0]	added
rxn12376_c0	<	(1) 4-Hydroxy-benzylalcohol_e0[e0] <= (1) 4-Hydroxy-benzylalcohol_c0[c0]	added
rxn12868_c0	>	(1) H2O_c0[c0] + (1) ATP_c0[c0] + (2) NAD_c0[c0] + (1) H2S_c0[c0] + (1) Dethiobiotin_c0[c0] => (2) NADH_c0[c0] + (1) PPi_c0[c0] + (1) AMP_c0[c0] + (3) H+_c0[c0] + (1) BIOT_c0[c0]	added
rxn05464_c0	>	(1) NADH_c0[c0] + (1) H+_c0[c0] + (1) trans-Octodec-2-enoyl-ACP_c0[c0] => (1) NAD_c0[c0] + (1) Octodecanoyl-ACP_c0[c0]	added
rxn00125_c0	<	(1) H2O_c0[c0] + (1) S-Adenosyl-L-methionine_c0[c0] <= (1) H+_c0[c0] + (1) 5-Methylthioadenosine_c0[c0] + (1) L-Homoserine_c0[c0]	added
rxn09310_c0	>	(1) ATP_c0[c0] + (1) L-Tyrosine_c0[c0] + (1) L-Cysteine_c0[c0] + (1) 1-deoxy-D-xylulose5-phosphate_c0[c0] => (1) H2O_c0[c0] + (1) CO2_c0[c0] + (1) PPi_c0[c0] + (1) AMP_c0[c0] + (1) L-Alanine_c0[c0] + (2) H+_c0[c0] + (1) 4-Methyl-5--2-phosphoethyl-thiazole_c0[c0] + (1) 4-Hydroxy-benzylalcohol_c0[c0]	added

In [10]:
model.medium = {'EX_pi_e': 1000.0,
# 'EX_co2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_cl_e': 1000.0,
# 'EX_o2_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

NameError: name 'model' is not defined

In [217]:
model.medium

{'EX_pi_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [ ]:
mo

In [264]:

biolog = Biolog()
biolog.add_plate(pm1_plate)

In [265]:
solution = biolog.run_plates(model)

skip asp__D
skip 2obut
skip tartr__M
skip rbt
skip datp
skip glx
skip tcb


In [266]:
pm1_plate

Negative Control,L-Arabinose,N-Acetyl-DGlucosamine,D-Saccharic Acid,Succinic Acid,D-Galactose,L-Aspartic Acid,L-Proline,D-Alanine,D-Trehalose,D-Mannose,Dulcitol
D-Serine,D-Sorbitol,Glycerol,L-Fucose,D-Glucuronic Acid,D-Gluconic Acid,"D,L-α-GlycerolPhosphate",D-Xylose,L-Lactic Acid,Formic Acid,D-Mannitol,L-Glutamic Acid
Glucose-6-Phosphate,D-Galactonic Acid-γ-Lactone,"D,L-Malic Acid",D-Ribose,Tween 20,L-Rhamnose,D-Fructose,Acetic Acid,α-D-Glucose,Maltose,D-Melibiose,Thymidine
L-Asparagine,D-Aspartic Acid,D-Glucosaminic Acid,"1,2-Propanediol",Tween 40,α-Keto-Glutaric Acid,α-Keto-Butyric Acid,α-Methyl-DGalactoside,α-D-Lactose,Lactulose,Sucrose,Uridine
L-Glutamine,M-Tartaric Acid,Glucose-1-Phosphate,Fructose-6-Phosphate,Tween 80,α-Hydroxy Glutaric Acid-γLactone,α-Hydroxy Butyric Acid,β-Methyl-DGlucoside,Adonitol,Maltotriose,2-Deoxy Adenosine,Adenosine
Glycyl-L-Aspartic Acid,Citric Acid,M-Inositol,D-Threonine,Fumaric Acid,Bromo Succinic Acid,Propionic Acid,Mucic Acid,Glycolic Acid,Glyoxylic Acid,D-Cellobiose,Inosine
Glycyl-L-Glutamic Acid,Tricarballylic Acid,L-Serine,L-Threonine,L-Alanine,L-Alanyl-Glycine,Acetoacetic Acid,N-Acetyl-β-D-Mannosamine,Mono Methyl Succinate,Methyl Pyruvate,D-Malic Acid,L-Malic Acid
Glycyl-L-Proline,P-Hydroxy Phenyl Acetic Acid,M-Hydroxy Phenyl Acetic Acid,Tyramine,D-Psicose,L-Lyxose,Glucuronamide,Pyruvic Acid,L-Galactonic Acid-γ-Lactone,D-Galacturonic Acid,Phenylethylamine,2-Aminoethano


In [233]:
solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.15741431904825262

In [117]:
model = cobra.io.read_sbml_model('iML1515.xml')

In [270]:
import cobra
from IPython.core.display import display, HTML

In [296]:
def s_repr_html_(plate):
        html = "<h3>" + plate.id + "</h3>"
        t = '<table style="border: 3px double black"><tbody>'
        for row in plate.rows:
            t += '<tr style="border: 3px double black">'
            for col in plate.cols:
                cell = '<td style="border: 3px double black;text-align:left;vertical-align:top;background-color:{}">{}</td>'
                color = 'white'
                for o in plate.wells:
                    if o == row + col:
                        well = plate.wells[o]
                        #print(well)
                        if 'growth' in well:
                            color = '#d3d3d3'
                            if well['growth'] > 0:
                                color = 'green'
                        cell = cell.format(color, str(well['growth'])  + '<br>' + plate.wells[o]['desc'])
                        #t += 
                        #print(self.wells[o])
                t += cell
            t += "</tr>"
        t += "</tbody></table>"
        html += t
        return html
HTML(s_repr_html_(pm1_plate))

nanNegative Control,0.09956267661964154L-Arabinose,0.1792556643679916N-Acetyl-DGlucosamine,0.10950305235956234D-Saccharic Acid,nanSuccinic Acid,0.13571995313752214D-Galactose,0.028394121225227196L-Aspartic Acid,nanL-Proline,nanD-Alanine,0.3743579781555464D-Trehalose,0.15741431904825157D-Mannose,0.11317075509602377Dulcitol
0.005162346409250346D-Serine,0.1140255872267935D-Sorbitol,0.005655671304743194Glycerol,0.06783522421665451L-Fucose,nanD-Glucuronic Acid,nanD-Gluconic Acid,"nanD,L-α-GlycerolPhosphate",0.09956267661963994D-Xylose,nanL-Lactic Acid,nanFormic Acid,0.11402558722678244D-Mannitol,0.028672897876102936L-Glutamic Acid
0.2224974167804391Glucose-6-Phosphate,nanD-Galactonic Acid-γ-Lactone,"0.0696808590581734D,L-Malic Acid",0.03447957888745262D-Ribose,nanTween 20,0.06783522421665454L-Rhamnose,0.15741431904825243D-Fructose,nanAcetic Acid,0.15741431904825123α-D-Glucose,0.37435797815554417Maltose,0.3526636122448197D-Melibiose,0.13500673243031636Thymidine
0.028502849188262867L-Asparagine,nanD-Aspartic Acid,nanD-Glucosaminic Acid,"nan1,2-Propanediol",nanTween 40,0.04213149078151659α-Keto-Glutaric Acid,nanα-Keto-Butyric Acid,nanα-Methyl-DGalactoside,0.35266361224481285α-D-Lactose,nanLactulose,nanSucrose,0.18980348505620737Uridine
0.028775800830833264L-Glutamine,nanM-Tartaric Acid,0.15741431904825157Glucose-1-Phosphate,0.22249741678041224Fructose-6-Phosphate,nanTween 80,nanα-Hydroxy Glutaric Acid-γLactone,nanα-Hydroxy Butyric Acid,nanβ-Methyl-DGlucoside,nanAdonitol,0.5913016372628709Maltotriose,nan2-Deoxy Adenosine,0.21906251408963706Adenosine
nanGlycyl-L-Aspartic Acid,0.0743303785705343Citric Acid,nanM-Inositol,nanD-Threonine,0.028394121225226693Fumaric Acid,nanBromo Succinic Acid,nanPropionic Acid,nanMucic Acid,nanGlycolic Acid,nanGlyoxylic Acid,nanD-Cellobiose,0.19541643316414903Inosine
nanGlycyl-L-Glutamic Acid,nanTricarballylic Acid,0.005162346409250675L-Serine,nanL-Threonine,nanL-Alanine,nanL-Alanyl-Glycine,0.02014432732097885Acetoacetic Acid,nanN-Acetyl-β-D-Mannosamine,nanMono Methyl Succinate,nanMethyl Pyruvate,nanD-Malic Acid,0.02839412122522747L-Malic Acid
nanGlycyl-L-Proline,nanP-Hydroxy Phenyl Acetic Acid,nanM-Hydroxy Phenyl Acetic Acid,nanTyramine,nanD-Psicose,0.09956267661964202L-Lyxose,nanGlucuronamide,0.0049926229039725266Pyruvic Acid,nanL-Galactonic Acid-γ-Lactone,nanD-Galacturonic Acid,nanPhenylethylamine,nan2-Aminoethano
